# ONNX to TensorRT and inference with TensorRT

This notebook demonstrates how to convert onnx model to TensorRT engine and then inference
with TensorRT engine.

We use [PaddlePaddle](https://github.com/PaddlePaddle/PaddleOCR) text detection model for demonstration
purpose. If you have any questions regarding the *model* itself, please create an issue in the original
paddlepaddle repo.

Please make sure you have run `pip install -r requirements.txt` both in this folder and the parent folder
to make sure all necessary packages are installed.

For installation and environment setup, please refer to [README](README.md).

In [ ]:
import sys
sys.path.append("../")
from trt_utils import TRTUtils

import cv2
from matplotlib import pyplot as plt
import numpy as np
from model_common import build_pre_process, build_post_process
from model_common.utils import viz_pddle_distill_db_results

## Inference using the TensoRT Engine

This part is the same as the normal onnx inference process, but with tensorRT inference
instead of onnxruntime

### Text Detection

Perform preprocessing steps

In [ ]:
test_img_path = "../sample_images/nutrition_label.jpg"
test_img = cv2.imread(test_img_path)
pre_process_op = build_pre_process({"name": "td_PaddlePaddlePreProcess"})
test_in, to_postprocess = pre_process_op(raw_img=test_img, dimension=(960,960))

Run inference with tensorRT

Important to use pycuda here. The autoinit will automatically initialize GPU device.
If not autoinit, an error will occur

In [ ]:
dynamic_trt_config = {
            "fp16":True,
            "original_model_path":  "../models/ch_PP-OCRv2_det_infer.onnx",
            "engine_path":"../models/ch_PP-OCRv2_det_infer.engine",
            "dynamic_shape": True,
            "profile_config":[
                {"x" : [(1, 3, 960, 960), (1, 3, 1280, 1280), (1, 3, 1536,1536) ] }
            ]
        }
static_trt_config = {
    "fp16":True,
    "original_model_path": "../models/ch_PP-OCRv2_det_infer.onnx",
    "engine_path":"../models/ch_PP-OCRv2_det_infer_static.engine",
    "dynamic_shape": False,
    "profile_config":[
        {"x" : [(1, 3, 1088, 1440)] }
    ]
}
trt_utils = TRTUtils(dynamic_trt_config)
trt_utils.get_engine()
output_buffer = trt_utils.inference_single(test_in, profiling=False)

In [ ]:
# reshape
output_shape = list(test_in.shape)
output_shape[1] = 1
output_shape = [1] + output_shape
results = np.reshape(output_buffer[0],output_shape)

Perform post-processing steps

In [ ]:
postprocess_params = {
    "name": "td_PaddlePaddlePostProcess",
    "thresh": 0.3,
    "box_thresh": 0.5,
    "max_candidate": 1000,
    "unclip_ratio": 2,
    "use_dilation": False,
    "score_mode": "fast",
}
post_process_op = build_post_process(postprocess_params)
dt_boxes = post_process_op(results=results, **to_postprocess)

Viz the prediction output

In [ ]:
viz_img = viz_pddle_distill_db_results(test_img.copy(), dt_boxes=dt_boxes)
plt.imshow(cv2.cvtColor(viz_img, cv2.COLOR_BGR2RGB))

In [ ]:
plt.clf()
plt.close()

### Text Recognition